In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from math import pi, floor

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from PIL import Image
from torchvision import transforms
import requests
from torch.autograd import Variable
import core
from fft import roll, fftshift, ifftshift
from torch.fft import ifft, fft, ifftn, fftn
from IQA_pytorch import SSIM, VIF
import gc

In [ ]:
torch.autograd.set_detect_anomaly(True)

attack_path = 'result/simulation/' # + "name_of_model/image"
slm_path = 'result/simulation/' # + "name_of_model/slm"

In [ ]:
GPU_NUM = 0 # GPU Num
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU

print ('Current cuda device ', torch.cuda.current_device()) # check

# Additional Infos
if device.type == 'cuda':
    print(torch.cuda.get_device_name(GPU_NUM))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')
    
gc.collect()
torch.cuda.empty_cache()

In [ ]:
## Set random seed for reproducibility
manualSeed = 999
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:
# Choose the pytorch model for which you want to generate adversarial attack

resnet50 = models.resnet50(pretrained=True)
resnet50.eval()
resnet50.to(device)

# vgg16 = models.vgg16(pretrained=True)
# vgg16.eval()
# vgg16.to(device)

# mobilenet = models.mobilenet_v3_large(pretrained=True)
# mobilenet.eval()
# mobilenet.to(device)

In [ ]:
## store data
data=np.zeros((1000,3,224,224)).astype('float32') # Reshape the data as per the input size of Models
labels=(np.zeros(1000))
labels=labels.astype(int)


## mean and std will remain same irresptive of the model you use
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

preprocess11 = transforms.Compose([
    transforms.Normalize(mean, std)])

def normalize(t):
    t[:, 0, :, :] = (t[:, 0, :, :] - mean[0])/std[0]
    t[:, 1, :, :] = (t[:, 1, :, :] - mean[1])/std[1]
    t[:, 2, :, :] = (t[:, 2, :, :] - mean[2])/std[2]

    return t


## readable labels (download from https://savan77.github.io/blog/imagenet_adv_examples.html)
labels_link = "https://savan77.github.io/blog/labels.json"
labels_json = requests.get(labels_link).json()
labells = {int(idx): label for idx, label in labels_json.items()}


## loading data
to_tensor = transforms.ToTensor()

to_pil = transforms.ToPILImage()

def torch_to_np(img_var):
    return img_var.detach().cpu().numpy()[0]

def load(path):
    img = Image.open(path)
    return img


## IQA
model1 = SSIM(channels=3)

def psnr(img1, img2):
    mse = torch.mean((img1 * 255 - img2 * 255) ** 2)
    return 20 * torch.log10(255.0 / torch.sqrt(mse))

In [ ]:
padsize = 1
m = 224  #Pixel number of SLM
factor = 2  #For Nyquist sampling
factor2 = 1
M = m * factor * factor2  #Image size
f = 100000  #Focal length of relay lens in um
length = [0.61, 0.53, 0.47]  #Wavelength in um [Red Green Blue]. Note that center wavelegth is green color.
length = np.array(length)
fov = 250000  #Field-of-view in um
eff_pixelsize = fov/M  #Effective pixel size
slm_pixelsize = 30  #Pixel size of SLM in um
slm_pixelnum = m  #추가

PhySizeSLM = slm_pixelnum * slm_pixelsize
MaxFreqSLM = PhySizeSLM / length / f / 2
DelFreqSLM = slm_pixelsize / np.min(length) / f  #기준은 가장 High frequency를 전달할 수 있는 Blue channel

kx = (DelFreqSLM * torch.range(-M / 2, M / 2 - 1)).to(torch.double)
ky = (DelFreqSLM * torch.range(-M / 2, M / 2 - 1)).to(torch.double)
kxm, kym = torch.meshgrid(kx, ky)

Ratio = MaxFreqSLM / MaxFreqSLM[2]  #Radius ratio of each color channel

cent = M/2
initial = torch.rand(224, 224).to(device)

In [ ]:
## Random perturbation

class FourierDomain(torch.nn.Module):
    def __init__(self):
        super(FourierDomain, self).__init__()
        self.MM = torch.nn.Parameter(initial)
        self.MM.requires_grad = True

    def forward(self, inputIntensityFT, attack):
        slm = self.MM
        
        slm = torch.squeeze(torch.squeeze(F.interpolate(torch.unsqueeze(torch.unsqueeze(slm, 0), 0), scale_factor=factor2)))
        CTF_B = (torch.sqrt(torch.pow(kxm, 2) + torch.pow(kym, 2)) < MaxFreqSLM[2] * factor2).to(torch.double)
        CTF_B = CTF_B.to(device)
        slm = slm * CTF_B[int(cent-M/4):int(cent+M/4),int(cent-M/4):int(cent+M/4)]  #Physicially displayed SLM pattern
        
        slm_R = Ratio[0] * torch.squeeze(torch.squeeze(F.interpolate(torch.unsqueeze(torch.unsqueeze(slm, 0), 0), scale_factor=Ratio[0])))
        slm_G = Ratio[1] * torch.squeeze(torch.squeeze(F.interpolate(torch.unsqueeze(torch.unsqueeze(slm, 0), 0), scale_factor=Ratio[1])))
        slm_B = slm
        
        slm_r = torch.zeros(M, M).to(device)
        slm_g = torch.zeros(M, M).to(device)
        slm_b = torch.zeros(M, M).to(device)
        
        x, y = tuple(list(slm_R.size()))
        slm_r[int(M / 2 + 1 + floor(-y / 2)):int(M / 2 + 1 + floor(y / 2)),
        int(M / 2 + 1 + floor(-x / 2)):int(M / 2 + 1 + floor(x / 2))] = slm_R

        x, y = tuple(list(slm_G.size()))
        slm_g[int(M / 2 + 1 + floor(-y / 2)):int(M / 2 + 1 + floor(y / 2)),
        int(M / 2 + 1 + floor(-x / 2)):int(M / 2 + 1 + floor(x / 2))] = slm_G

        x, y = tuple(list(slm_B.size()))
        slm_b[int(M / 2 + 1 + floor(-y / 2)):int(M / 2 + 1 + floor(y / 2)),
        int(M / 2 + 1 + floor(-x / 2)):int(M / 2 + 1 + floor(x / 2))] = slm_B
        
        CTF_R = (slm_r != 0).to(torch.double)
        CTF_G = (slm_g != 0).to(torch.double)
        CTF_B = (slm_b != 0).to(torch.double)
        
        atk_CTF_R_mask_stack = torch.cat([torch.unsqueeze(torch.cos(slm_r), 2), torch.unsqueeze(torch.sin(slm_r), 2)],
                                         dim=2)
        atk_CTF_R_mask = torch.view_as_complex(atk_CTF_R_mask_stack)
        atk_CTF_R = CTF_R * atk_CTF_R_mask

        atk_CTF_G_mask_stack = torch.cat([torch.unsqueeze(torch.cos(slm_g), 2), torch.unsqueeze(torch.sin(slm_g), 2)],
                                         dim=2)
        atk_CTF_G_mask = torch.view_as_complex(atk_CTF_G_mask_stack)
        atk_CTF_G = CTF_G * atk_CTF_G_mask

        atk_CTF_B_mask_stack = torch.cat([torch.unsqueeze(torch.cos(slm_b), 2), torch.unsqueeze(torch.sin(slm_b), 2)],
                                         dim=2)
        atk_CTF_B_mask = torch.view_as_complex(atk_CTF_B_mask_stack)
        atk_CTF_B = CTF_B * atk_CTF_B_mask
        
        if attack == True:
            ft_atk_CTF_R = fftn(fftshift(atk_CTF_R))
            MTF_R = ifftshift(ifftn((ft_atk_CTF_R * torch.conj(ft_atk_CTF_R))))

            ft_atk_CTF_G = fftn(fftshift(atk_CTF_G))
            MTF_G = ifftshift(ifftn((ft_atk_CTF_G * torch.conj(ft_atk_CTF_G))))

            ft_atk_CTF_B = fftn(fftshift(atk_CTF_B))
            MTF_B = ifftshift(ifftn((ft_atk_CTF_B * torch.conj(ft_atk_CTF_B))))

        elif attack == False:
            ft_CTF_R = fftn(fftshift(CTF_R))
            MTF_R = ifftshift(ifftn((ft_CTF_R * torch.conj(ft_CTF_R))))

            ft_CTF_G = fftn(fftshift(CTF_G))
            MTF_G = ifftshift(ifftn((ft_CTF_G * torch.conj(ft_CTF_G))))

            ft_CTF_B = fftn(fftshift(CTF_B))
            MTF_B = ifftshift(ifftn((ft_CTF_B * torch.conj(ft_CTF_B))))

        MTF_R = MTF_R / torch.max(torch.max(torch.abs(MTF_R)))
        MTF_G = MTF_G / torch.max(torch.max(torch.abs(MTF_G)))
        MTF_B = MTF_B / torch.max(torch.max(torch.abs(MTF_B)))

        #inputIntensityFT = torch.as_tensor(inputIntensityFT, dtype=torch.cdouble)
        inputIntensityFT_R = inputIntensityFT[:, :, 0]
        inputIntensityFT_G = inputIntensityFT[:, :, 1]
        inputIntensityFT_B = inputIntensityFT[:, :, 2]
        
        outputFT_R = MTF_R * inputIntensityFT_R
        outputFT_G = MTF_G * inputIntensityFT_G
        outputFT_B = MTF_B * inputIntensityFT_B
        
        ifft_R = torch.abs(ifftn(ifftshift(outputFT_R))).to(torch.float32) / 255
        ifft_G = torch.abs(ifftn(ifftshift(outputFT_G))).to(torch.float32) / 255
        ifft_B = torch.abs(ifftn(ifftshift(outputFT_B))).to(torch.float32) / 255
                
        output_R = torch.squeeze(torch.squeeze(F.interpolate(torch.unsqueeze(torch.unsqueeze(ifft_R, 0), 0), size=(224, 224))))
        output_G = torch.squeeze(torch.squeeze(F.interpolate(torch.unsqueeze(torch.unsqueeze(ifft_G, 0), 0), size=(224, 224))))
        output_B = torch.squeeze(torch.squeeze(F.interpolate(torch.unsqueeze(torch.unsqueeze(ifft_B, 0), 0), size=(224, 224))))
    
        image_tensor2 = torch.stack([output_R, output_G, output_B])
        
    
        return image_tensor2, slm

In [ ]:
images = pd.read_csv("images.csv") 
path= 'clean_images' # download from https://kaggle.com/c/6864

for i in range(1000):
    ImgID= images.iloc[i,0] 
    img = load(path +'/'+ ImgID + '.png')
    img = img.resize((224,224), Image.ANTIALIAS) # Change the input image shape here
    img = to_tensor(img)[None, :]
    
    img_np = torch_to_np(img)
    data[i,:,:,:] = img_np
    labels[i]= images.iloc[i,6]

In [ ]:
for i in range(1000):
    
    ImgID= images.iloc[i,0] 
    img= data[i]
    img= torch.from_numpy(img)[None,:]
    label= labels[i]-1 # the first label is redundant
    label= torch.tensor([label] , dtype=torch.int64)
    img, label = img.to(device), label.to(device)
    
    IMG2 = core.imresize(img, sizes=(M, M))
    inputIntensityFT_R = fftshift(fftn(IMG2[:,0,:,:])).squeeze(0)
    inputIntensityFT_G = fftshift(fftn(IMG2[:,1,:,:])).squeeze(0)
    inputIntensityFT_B = fftshift(fftn(IMG2[:,2,:,:])).squeeze(0)
    inputIntensityFT = torch.stack([inputIntensityFT_R, inputIntensityFT_G, inputIntensityFT_B])
    inputIntensityFT = (inputIntensityFT * 255).permute(1, 2, 0)
    inputIntensityFT = inputIntensityFT.detach().to(device)
    
    initial = torch.rand(224, 224).to(device)
    model = FourierDomain()
    model.to(device)
    
    image_tensor, slm = model(inputIntensityFT, attack=False)
    image_tensor = preprocess11(image_tensor)  # preprocess an i
    image_tensor = image_tensor.unsqueeze(0)  # add batch dimension.  C X H X W ==> B X C X H X W
    image_tensor = image_tensor.detach()
    del initial, model, slm
    
    x1 = image_tensor.squeeze(0)
    x1 = x1.mul((torch.FloatTensor(std)).to(device).view(3, 1, 1)).add(
        (torch.FloatTensor(mean)).to(device).view(3, 1, 1)).cpu().detach().numpy()  # reverse of normalization op- "unnormalize"
    x1 = np.transpose(x1, (1, 2, 0))  # C X H X W  ==>   H X W X C
    x1 = np.clip(x1, 0, 1)
    ref = torch.as_tensor(x1).permute(2, 0, 1).unsqueeze(0)
    
    target = Variable(torch.LongTensor([label.item()]).to(device), requires_grad=False)
    
    x_adv_pred = labells[label.item()]
    param = 1e-2
    n_epoch = 300
    a = 0
    
    while x_adv_pred == labells[label.item()]:
        
        initial = torch.rand(224, 224).to(device)
        model = FourierDomain()
        model.to(device)
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=5 * (1e-3), weight_decay=5e-6)
        
        for epoch in range(0, n_epoch):

            optimizer.zero_grad()

            image_tensor2, slm = model(inputIntensityFT, attack=True)
            image_tensor2 = preprocess11(image_tensor2)
            image_tensor2 = image_tensor2.unsqueeze(0)
            output2 = mobilenet.forward(image_tensor2)
            
            # perform a backward pass in order to get gradients
            loss2 = param * torch.norm(image_tensor2 - image_tensor) - criterion(output2, target)
            
            x2 = image_tensor2.squeeze(0)
            x2 = x2.mul((torch.FloatTensor(std)).to(device).view(3, 1, 1)).add(
                (torch.FloatTensor(mean)).to(device).view(3, 1, 1)).cpu().detach().numpy()  # reverse of normalization op- "unnormalize"
            x2 = np.transpose(x2, (1, 2, 0))  # C X H X W  ==>   H X W X C
            x2 = np.clip(x2, 0, 1)
            dist = torch.as_tensor(x2).permute(2, 0, 1).unsqueeze(0)
            
            pp = psnr(dist, ref).item()
            ss = model1(dist, ref, as_loss=False).item()
            
            im = Image.fromarray(np.uint8(x2 * 255)).convert('RGB')
            imm = to_tensor(im)[None, :]
            imm = imm.to(device)
            output2 = mobilenet.forward(preprocess11(imm))
            x_adv_pred = labells[torch.max(output2.data, 1)[1][0].item()]
            
            if x_adv_pred != labells[label.item()]:
                break

            loss2.backward()
            optimizer.step()
    
        if x_adv_pred != labells[label.item()]:
            break
        
        del initial, model, criterion, optimizer, loss2
        
        param /= 10
        a += 1
        
        if a == 6:
            break
    
    print(i, ' ', param, ' ', pp, ' ', ss)
    im.save(attack_path+ImgID+'.png')
    MM = slm.cpu().detach().numpy()
    df = pd.DataFrame(MM)
    df.to_csv(slm_path+ImgID+'.csv', header=None, index=False)